In [6]:
import pandas as pd
import random
from datetime import datetime, timedelta
import os

# Temizlik personeli ID'lerini oluşturma
def create_personnel_ids(num_personnel):
    return [f"P{str(i).zfill(3)}" for i in range(1, num_personnel + 1)]

# Mini bar görevlisi personel ID'lerini oluşturma
def create_mini_bar_ids(num_personnel):
    return [f"M{str(i).zfill(3)}" for i in range(1, num_personnel + 1)]

# Arıza personeli ID oluşturma
def create_repair_ids(num_personnel):
    return [f"A{str(i).zfill(3)}" for i in range(1, num_personnel + 1)]

# Oda numaralarını oluşturma
def create_room_numbers(start, end):
    return [str(i) for i in range(start, end + 1)]

# Mini bar süresi belirleme
def get_mini_bar_duration():
    return random.randint(3, 6)  # 3-6 dakika arasında rastgele

# Arıza süresi belirleme
def get_repair_duration():
    return random.randint(10, 59)  # 10-59 dakika arasında rastgele

# Temizlik süresi belirleme
def get_cleaning_duration(cleaning_type, room_type, sheets_changed):
    if cleaning_type == "Rutin":
        if room_type == "Standart":
            return random.randint(5, 15) 
        elif room_type == "Suit":
            return random.randint(10, 15) 
        elif room_type == "Lüx Suit":
            return random.randint(15, 20) 
            
    elif cleaning_type == "Detaylı":
        if room_type == "Standart":
            return random.randint(20, 40) 
        elif room_type == "Suit":
            return random.randint(30, 50) 
        elif room_type == "Lüx Suit":
            return random.randint(40, 60) 
            
    elif cleaning_type == "Müşteri Talebi":
        return random.randint(5, 30)

# Otelin yıllık doluluk oranlarını belirleme
def get_occupancy_rate(month):
    if month in [6, 7, 8]:  # Yaz ayları (Haziran, Temmuz, Ağustos) -> %90-%100 doluluk
        return random.uniform(0.9, 1.0)
    elif month in [5, 9]:  # Bahar ayları (Mayıs, Eylül) -> %50-%70 doluluk
        return random.uniform(0.5, 0.7)
    elif month in [4, 10]:  # Geçiş ayları (Nisan, Ekim) -> %30-%40 doluluk
        return random.uniform(0.3, 0.4)
    else:  # Kış ayları (Kasım - Mart) -> %10-%30 doluluk
        return random.uniform(0.05, 0.1)

# Giriş saatlerini belirleme
def get_entry_time(task_type):
    if task_type == "Rutin":
        start_hour, end_hour = 8, 10
    elif task_type == "Detaylı":
        start_hour, end_hour = 12, 14
    elif task_type == "Mini Bar":
        start_hour, end_hour = 9, 11
    else:  # Müşteri talebi ve Arıza
        start_hour, end_hour = 8, 18

    random_hour = random.randint(start_hour, end_hour)
    random_minute = random.randint(0, 59)
    return datetime.strptime(f"{random_hour}:{random_minute:02d}", "%H:%M")

# **Personel çakışmasını önlemek için sözlük ekleniyor**
personnel_busy_times = {}

# **Meşgul personel için uygun giriş saatini belirleme**
def get_available_entry_time(personnel_id, task_type):
    new_entry_time = get_entry_time(task_type)

    if personnel_id in personnel_busy_times:
        last_end_time = personnel_busy_times[personnel_id]
        if new_entry_time < last_end_time:
            new_entry_time = last_end_time + timedelta(minutes=5)  # 5 dakika boşluk bırak

    return new_entry_time

# Odaların listesi
rooms = []
personnel_ids = create_personnel_ids(100)
mini_bar_ids = create_mini_bar_ids(50)
repair_ids = create_repair_ids(10)

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
current_date = start_date
room_cleaning_tracker = {} # Hangi odada kaç gün rutin temizlik yapıldığını takip eder

while current_date <= end_date:
    month = current_date.month
    occupancy_rate = get_occupancy_rate(month)

    mini_bar_index = 0
    for floor in range(1, 19):
        max_rooms = 70 if floor <= 10 else 50 if floor <= 14 else 25
        room_numbers = create_room_numbers(floor * 100 + 1, floor * 100 + max_rooms)
        personnel_per_floor = personnel_ids[(floor - 1) * 5: floor * 5]
        mini_bar_per_floor = mini_bar_ids[mini_bar_index: mini_bar_index + 2] if floor <= 14 else [mini_bar_ids[mini_bar_index]]
        mini_bar_index += len(mini_bar_per_floor)

        occupied_rooms = int(len(room_numbers) * occupancy_rate)
        random.shuffle(room_numbers)
        occupied_room_numbers = room_numbers[:occupied_rooms]

        for room in occupied_room_numbers:
            personnel_id = random.choice(personnel_per_floor)
            mini_bar_personnel = random.choice(mini_bar_per_floor)
            stay_duration = random.randint(1, 7)  # müşteriler min 1 max 7 gün konaklasınlar
            routine_days = room_cleaning_tracker.get((room, "Rutin"), 0)
            repair_personnel = random.choice(repair_ids)

            if routine_days >= stay_duration:
                cleaning_type = "Detaylı"
                room_cleaning_tracker[(room, "Rutin")] = 0  # Rutin sayacını sıfırla
            else:
                cleaning_type = "Rutin"
                room_cleaning_tracker[(room, "Rutin")] = routine_days + 1

            entry_time = get_available_entry_time(personnel_id, "Rutin")
            duration = get_cleaning_duration("Rutin", "Standart", "Evet")
            exit_time = entry_time + timedelta(minutes=duration)

            personnel_busy_times[personnel_id] = exit_time  # Çakışmayı önlemek için kaydet

            rooms.append({
                "Tarih": current_date.strftime("%Y-%m-%d"),
                "Oda Numarası": room,
                "Oda Tipi": "Standart" if floor <= 10 else "Suit" if floor <= 14 else "Lüx Suit",
                "Personel ID": personnel_id,
                "Temizlik Türü": cleaning_type,
                "Giriş Saati": entry_time.strftime("%H:%M"),
                "Çıkış Saati": exit_time.strftime("%H:%M")
            })

            # Mini Bar işlemi
            mini_bar_start_time = get_available_entry_time(mini_bar_personnel, "Mini Bar")
            mini_bar_duration = get_mini_bar_duration()
            mini_bar_end_time = mini_bar_start_time + timedelta(minutes=mini_bar_duration)

            personnel_busy_times[mini_bar_personnel] = mini_bar_end_time

            rooms.append({
                "Tarih": current_date.strftime("%Y-%m-%d"),
                "Oda Numarası": room,
                "Oda Tipi": "Standart" if floor <= 10 else "Suit" if floor <= 14 else "Lüx Suit",
                "Personel ID": mini_bar_personnel,
                "Temizlik Türü": "Mini Bar",
                "Giriş Saati": mini_bar_start_time.strftime("%H:%M"),
                "Çıkış Saati": mini_bar_end_time.strftime("%H:%M")
            })
            
            # Arıza işlemi
            if random.random() < 0.01:  
                repair_start_time = get_available_entry_time(repair_personnel, "Arıza")
                repair_duration = get_repair_duration()
                repair_end_time = repair_start_time + timedelta(minutes=repair_duration)

                rooms.append({
                    "Tarih": current_date.strftime("%Y-%m-%d"),
                    "Oda Numarası": room,
                    "Oda Tipi": "Standart" if floor <= 10 else "Suit" if floor <= 14 else "Lüx Suit",
                    "Personel ID": repair_personnel,
                    "Temizlik Türü": "Arıza",
                    "Giriş Saati": repair_start_time.strftime("%H:%M"),
                    "Çıkış Saati": repair_end_time.strftime("%H:%M"),
                })

    current_date += timedelta(days=1)

# Dosya ismi için tarih ve saat formatı belirleme
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename_csv = f"otel_temizlik_veriseti_{now}.csv"
filename_xlsx = f"otel_temizlik_veriseti_{now}.xlsx"

# DataFrame oluşturma
df = pd.DataFrame(rooms)

# Veri setini kaydetme
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
df.to_csv(os.path.join(desktop_path, filename_csv), index=False)
df.to_excel(os.path.join(desktop_path, filename_xlsx), index=False)

print(f"Güncellenmiş yıllık veri seti başarıyla oluşturuldu ve masaüstüne kaydedildi: \n{filename_csv}\n{filename_xlsx}")


Güncellenmiş yıllık veri seti başarıyla oluşturuldu ve masaüstüne kaydedildi: 
otel_temizlik_veriseti_2025-03-25_14-04-23.csv
otel_temizlik_veriseti_2025-03-25_14-04-23.xlsx
